This notebook runs simple, TVD, HAND, FwDET flood models for a set of reaches (defined by the class DefinitionsGenerator which can be specified in the .env file). The outputs are then compared against hydraulic model outputs to produce a set of comparison grids.

The total time to completion of 26 MDB scenes is around 16 hours (requires around 32GB ram)

Create folders if they do not exist

In [ ]:
from pathlib import Path
import os
from dotenv import load_dotenv
load_dotenv()

output_path_root = os.environ['OUTPUT_FOLDER_ROOT']
print(output_path_root)
Path(output_path_root +
     f"{os.path.sep}simple").mkdir(parents=True, exist_ok=True)
Path(output_path_root +
     f"{os.path.sep}hands").mkdir(parents=True, exist_ok=True)
Path(output_path_root + f"{os.path.sep}tvd").mkdir(parents=True, exist_ok=True)
Path(output_path_root +
     f"{os.path.sep}fwdet").mkdir(parents=True, exist_ok=True)
Path(output_path_root +
     f"{os.path.sep}tvd").mkdir(parents=True, exist_ok=True)


Create FWDET outputs

In [ ]:
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
from hydrological_connectivity.processing.fwdet_task import FwdetTask
from hydrological_connectivity.processing.compare_flood_rasters_rasterio import CompareFloodRastersRasterIo
from hydrological_connectivity.processing.compare_flood_rasters_elev_rasterio import CompareFloodRastersElevRasterIo
import logging
logging.getLogger().setLevel('INFO')

definition = DefinitionsGeneratorFactory.get_generator()
definition.generate()
tasks = []
for fwdet_output in definition.fwdet_outputs:
    try:
        if (fwdet_output.exists('all')):
            logging.info(f"Already exists: {fwdet_output}")
            continue
        logging.info(f"Processing: {fwdet_output}")
        st = FwdetTask(fwdet_output)
        st.execute()

        hydr_model_output = fwdet_output.hydraulic_model.elevation_outputs[
            fwdet_output.simulation_timespan['peak-event']]
        compare_water_height = CompareFloodRastersElevRasterIo(
            hydr_model_output,
            fwdet_output.output_path,
            fwdet_output.dem_path,
            fwdet_output.region_of_interest_albers,
            fwdet_output.output_path.replace(".tif", "_comparison_elev.tif"))
        compare_water_height.execute()

        hydr_model_output = fwdet_output.hydraulic_model.depth_outputs[
            fwdet_output.simulation_timespan['peak-event']]
        logging.info(f"truth - {hydr_model_output}")
        compare_water_height = CompareFloodRastersRasterIo(
            hydr_model_output,
            fwdet_output.output_path,
            fwdet_output.region_of_interest_albers,
            fwdet_output.output_path.replace(".tif", "_comparison.tif"))
        compare_water_height.execute()
    except:
        logging.exception(f"Raised an error with {fwdet_output}")
        # st.save_input_to_disk()


Create TVD outputs

In [ ]:
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
from hydrological_connectivity.processing.compare_flood_rasters_rasterio import CompareFloodRastersRasterIo
from hydrological_connectivity.processing.compare_flood_rasters_elev_rasterio import CompareFloodRastersElevRasterIo
from hydrological_connectivity.processing.tvd_task import TvdTask
import logging
logging.getLogger().setLevel('INFO')
definition = DefinitionsGeneratorFactory.get_generator()
definition.generate()
tasks = []
for tvd_output in definition.tvd_outputs:
    try:
        if (tvd_output.exists('ind')):
            logging.info(f"Already exists: {tvd_output}")
            continue
        logging.info(f"Processing: {tvd_output}")
        st = TvdTask(tvd_output)
        st.execute()

        hydr_model_output = tvd_output.hydraulic_model.elevation_outputs[
            tvd_output.simulation_timespan['peak-event']]
        compare_water_height = CompareFloodRastersElevRasterIo(
            hydr_model_output,
            tvd_output.output_path.replace(".tif", "_all.tif"),
            tvd_output.dem_path,
            tvd_output.region_of_interest_albers,
            tvd_output.output_path.replace(".tif", "_all_comparison_elev.tif"))
        compare_water_height.execute()
        compare_water_height = CompareFloodRastersElevRasterIo(
            hydr_model_output,
            tvd_output.output_path.replace(".tif", "_ind.tif"),
            tvd_output.dem_path,
            tvd_output.region_of_interest_albers,
            tvd_output.output_path.replace(".tif", "_ind_comparison_elev.tif"))
        compare_water_height.execute()

        hydr_model_output = tvd_output.hydraulic_model.depth_outputs[
            tvd_output.simulation_timespan['peak-event']]
        compare_water_height = CompareFloodRastersRasterIo(
            hydr_model_output,
            tvd_output.output_path.replace(".tif", "_all.tif"),
            tvd_output.region_of_interest_albers,
            tvd_output.output_path.replace(".tif", "_all_comparison.tif"))
        compare_water_height.execute()
        compare_water_height = CompareFloodRastersRasterIo(
            hydr_model_output,
            tvd_output.output_path.replace(".tif", "_ind.tif"),
            tvd_output.region_of_interest_albers,
            tvd_output.output_path.replace(".tif", "_ind_comparison.tif"))
        compare_water_height.execute()
    except:
        logging.exception(f"Raised an error with {tvd_output}")


Create hand outputs for different accumulation thresholds

In [ ]:
from hydrological_connectivity.processing.hand_task import HandTask
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
from hydrological_connectivity.processing.compare_flood_rasters_elev_rasterio import CompareFloodRastersElevRasterIo
from hydrological_connectivity.processing.compare_flood_rasters_rasterio import CompareFloodRastersRasterIo
import logging
logging.getLogger().setLevel('INFO')

definition = DefinitionsGeneratorFactory.get_generator()
definition.generate()
tasks=[]
accumulation_thresholds = [3000000, 1000000, 500000, 100000]
for hand_output in definition.hand_outputs:  
    for accumulation_threshold in accumulation_thresholds:
        try:
            if (hand_output.exists(accumulation_threshold, 'dep_comparison_elev')):
                logging.info(f"Already exists: {hand_output}")
                continue
            logging.info(f"Processing: {hand_output}")
            ht = HandTask(hand_output)
            ht.accumulation_threshold = accumulation_threshold
            ht.execute()
            
            hydr_model_output = hand_output.hydraulic_model.elevation_outputs[
                hand_output.simulation_timespan['peak-event']]
            compare_water_height = CompareFloodRastersElevRasterIo(
                hydr_model_output,
                hand_output.output_path.replace(
                    ".tif", f"_dep_{str(accumulation_threshold)}.tif"),
                hand_output.dem_path,
                hand_output.region_of_interest_albers,
                hand_output.output_path.replace(".tif", f"_dep_comparison_elev_{str(accumulation_threshold)}.tif"))
            compare_water_height.execute()

            hydr_model_output = hand_output.hydraulic_model.depth_outputs[
                hand_output.simulation_timespan['peak-event']]
            compare_water_height = CompareFloodRastersRasterIo(
                hydr_model_output,
                hand_output.output_path.replace(
                    ".tif", f"_dep_{str(accumulation_threshold)}.tif"),
                hand_output.region_of_interest_albers,
                hand_output.output_path.replace(".tif", f"_dep_comparison_{str(accumulation_threshold)}.tif"))
            compare_water_height.execute()
        except:
            logging.exception(f"Raised an error with {hand_output}")
